In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('./data/session.csv',encoding='gb18030')

In [3]:
data.head()

,用户ID,会话开始时间,咨询类型,咨询总结备注,详情
0,31964968,2019-05-23 23:55:11,售前问题（在线专用）->商品咨询 - -,nan,机器人2(蜜芽智能客服) 2019-05-23 23:55:11\n[xh]甄选洗衣液销售火...
1,32860495,2019-05-23 23:54:58,售中问题（在线专用）->物流问题 - -,nan,机器人4(蜜芽智能客服) 2019-05-23 23:54:58\n[xh]甄选洗衣液销售火...
2,24009062,2019-05-23 23:54:03,售中问题（在线专用）->物流问题 - -,nan,机器人3(蜜芽智能客服) 2019-05-23 23:54:03\n[xh]甄选洗衣液销售火...
3,34644663,2019-05-23 23:53:11,售前问题（在线专用）->商品咨询 - -,nan,机器人2(蜜芽智能客服) 2019-05-23 23:53:11\n[xh]甄选洗衣液销售火...
4,38445693,2019-05-23 23:53:02,售中问题（在线专用）->取消订单 - -,nan,机器人1(蜜芽智能客服) 2019-05-23 23:53:02\n[xh]甄选洗衣液销售火...


In [4]:
data[data['用户ID']=='31964968']['详情']

0    机器人2(蜜芽智能客服) 2019-05-23 23:55:11\n[xh]甄选洗衣液销售火...
Name: 详情, dtype: object

In [5]:
data['咨询类型'].unique()

array(['售前问题（在线专用）->商品咨询 - -', '售中问题（在线专用）->物流问题 - -',
       '售中问题（在线专用）->取消订单 - -', '售后问题（在线专用）->无理由退换货 - -',
       '售后问题（在线专用）->少件 - -', '售后问题（在线专用）->技术问题 - -',
       '售前问题（在线专用）->活动咨询浦发银行 - -', '临时问题（在线专用）->临时问题 - -',
       '规则问题（商管专用）->会员账户 - -', '售前问题（在线专用）->活动咨询（主站活动咨询） - -',
       '售前问题（在线专用）->政策咨询 - -', 'plus分销->PLUS会员提现问题 - -',
       'plus分销->PLUS会员政策咨询 - -', '售后问题（在线专用）->质量问题 - -',
       '售中问题（在线专用）->修改订单 - -', '售后问题（在线专用）->页面与实物不符 - -',
       '售后问题（在线专用）->商品使用咨询 - -', '售后问题（在线专用）->催退款 - -',
       '无效咨询（在线专用）->无效咨询 - -', '社群专用->订单问题（售后问题相关） - -',
       '社群专用->小寰基础操作（登录、修改资料等） - -', '入驻问题（商管专用）->入驻咨询 - -',
       '售前问题（在线专用）->活动咨询（plus问题咨询） - -', 'plus分销->PLUS技术问题 - -',
       '售后问题（在线专用）->发错货 - -', '临时问题（在线专用）->解绑手机 - -',
       '售后问题（在线专用）->保价 - -', '售中问题（在线专用）->催发货 - -',
       '售前问题（在线专用）->活动咨询话费充值 - -', 'plus分销->PLUS大礼包相关 - -',
       '售后问题（在线专用）->破损 - -', '售后问题（在线专用）->发票 - -',
       '社群专用->账户及收益（查销售额、升级业绩等） - -', '售后问题（在线专用）->合作问题 - -',
       '社群专用->售前咨询 - -'

In [6]:
basic_type = data['咨询类型'].str.split('->', n=1, expand=True)

In [7]:
data['基本咨询类型'] = basic_type[0]

In [8]:
data['基本咨询类型'].unique()

array(['售前问题（在线专用）', '售中问题（在线专用）', '售后问题（在线专用）', '临时问题（在线专用）',
       '规则问题（商管专用）', 'plus分销', '无效咨询（在线专用）', '社群专用', '入驻问题（商管专用）',
       '商品问题（商管专用）', '订单问题（商管专用）', '财务问题（商管专用）', '店铺问题（商管专用）',
       '售后问题（商管专用）', '默认分组', '售后问题（蜜店宝）', '结算问题（商管专用）'], dtype=object)

In [9]:
class_mapping = {label:idx for idx,label in enumerate(set(data['基本咨询类型']))}
data['label'] = data['基本咨询类型'].map(class_mapping)

In [10]:
class_mapping

{'售前问题（在线专用）': 0,
 '规则问题（商管专用）': 1,
 '社群专用': 2,
 '订单问题（商管专用）': 3,
 '售后问题（商管专用）': 4,
 '结算问题（商管专用）': 5,
 '入驻问题（商管专用）': 6,
 '商品问题（商管专用）': 7,
 '默认分组': 8,
 '无效咨询（在线专用）': 9,
 '店铺问题（商管专用）': 10,
 '售后问题（蜜店宝）': 11,
 'plus分销': 12,
 '财务问题（商管专用）': 13,
 '售中问题（在线专用）': 14,
 '临时问题（在线专用）': 15,
 '售后问题（在线专用）': 16}

In [11]:
data['label'].unique()

array([ 0, 14, 16, 15,  1, 12,  9,  2,  6,  7,  3, 13, 10,  4,  8, 11,  5])

In [12]:
len(data['label'].unique())

17

In [13]:
data['detail'] = data['详情'].str.replace('\n', ' ')

In [14]:
details = data['详情'].str.split('\n', expand=True)

In [15]:
details.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462
0,机器人2(蜜芽智能客服) 2019-05-23 23:55:11,[xh]甄选洗衣液销售火爆，正在陆续发货中，预计【26日前全部发完】，好货值得您的期待，感谢...,,[xh]亲亲，电影票活动已经上线啦，活动链接： https://mp.weixin.qq.c...,31964968 2019-05-23 23:55:19,转人工,机器人2(蜜芽智能客服) 2019-05-23 23:55:32,正在为您转接人工客服，请稍等...!,孙振伟(蜜芽客服伟伟) 2019-05-23 23:55:33,亲爱的plus会员，很高兴为您服务，麻烦您简述订单和问题，我为您查询处理。（温馨提示：您已进...,机器人2(蜜芽智能客服) 2019-05-23 23:55:33,已成功为您转接人工客服！,孙振伟(蜜芽客服伟伟) 2019-05-23 23:55:38,您好，非常荣幸为您服务！请问您遇到了什么问题？,31964968 2019-05-23 23:56:02,你好 咱们合作的信用卡可以在医院走VIP吗？,孙振伟(蜜芽客服伟伟) 2019-05-23 23:56:38,非常抱歉，亲亲，这个好像是不能的,对话结束 >>,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [16]:
def get_first_sentence(sentences):
    first_sentence = ''
    for i in range(len(sentences)):
        if sentences[i] is not None and '2019-05-23' in sentences[i] and '蜜芽' not in sentences[i]:
            if len(sentences[i+1]) > 3 and '人工客服' not in sentences[i+1] and '在线咨询' not in sentences[i+1] and '发送图片' not in sentences[i+1]:
                first_sentence += sentences[i+1]+' '
                # break  # only first sentence
    return first_sentence

In [17]:
first_sentences = details.apply(lambda row: get_first_sentence(row), axis=1)

In [18]:
first_sentences.head()

0                              你好 咱们合作的信用卡可以在医院走VIP吗？ 
1            你们快递 今天还没到 快递太吓人 已经8天了还没到货 为什么这么慢 这个不好解释 
2    你好，我换货的商品已经寄回了，快递也显示签收了，什么时候寄回给我 退回的商品是否收到 我是要...
3    帮我查一下这一款能不能链接wifl https://m.mia.com/item-33626...
4    没有显示取消 取消订单 没有显示取消选项呢 订单号：1905231956018268 张敏艳...
dtype: object

In [19]:
data['first_sentence'] = first_sentences

In [23]:
data = data[data['first_sentence'] != '']
train, dev = train_test_split(data, test_size=0.1, random_state=2019)

In [24]:
print(train.shape, dev.shape)  # test.shape, 

(4250, 9) (473, 9)


In [25]:
train[['label', 'first_sentence']].to_csv('data/mytask/train.tsv', index=False, header=False, sep='\t')
#test[['label', 'first_sentence']].to_csv('data/mytask/test.tsv', index=False, header=False, sep='\t')
dev[['label', 'first_sentence']].to_csv('data/mytask/dev.tsv', index=False, header=False, sep='\t')